<span style='color:gray'> Author: Glicerio Vergara.</span>

*https://www.linkedin.com/in/glicerio-vergara*

# <span style='color:DarkBlue'> Comparing two companies </span>  
___

Using the Dash framework, the Python script below compares the financial performance of two companies in a user-friendly web-based interface. The main objective of the code is to create a tool that allows users to input a list of stock tickers, retrieve relevant financial information for each company, and present a side-by-side visual and numerical comparison of key performance indicators (KPIs). The tool fetches data using Yahoo Finance API, processes the data, converts non-USD currencies to USD, and formats large numbers for market capitalization and revenue. The comparison includes various financial metrics such as market capitalization, revenue, earnings growth, dividend yield, profit margins, EBITDA margins, and P/E ratio. The user interface is designed with Dash, providing a visually appealing and interactive layout for easy interpretation. Additionally, the tool retrieves and displays company logos for visual identification. In sumarry, this code serves as a versatile financial analysis tool for investors or analysts interested in comparing the performance of two companies in a specific market or industry.

### <span style='color:DarkBlue'>  Import the libraries </span>  

In [7]:
import yfinance as yf
from forex_python.converter import CurrencyRates
import pandas as pd
import numpy as np
from urllib.parse import urlparse
import requests
from PIL import Image
from io import BytesIO

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

### <span style='color:DarkBlue'>  ETL funtions </span>  

In [14]:
def convert_to_usd(row):
    if row['currency'] != 'USD':
        c = CurrencyRates()
        currency=row['currency']
        exchange_rate = c.get_rate(currency, 'USD')
        row['revenue'] *= exchange_rate
        row['market_cap'] *= exchange_rate
    return row

def transform_all_except_specified(df, exclude_columns):
    for column in df.columns:
        if column not in exclude_columns:
            df[column] = pd.to_numeric(df[column], errors='coerce')
    return df

# Function to format decimal subscription
def format_decimal_subscript(data):
    if data >= 1e12:  # Trillion
        return f"${data / 1e12:.2f} T"
    elif data >= 1e9:  # Billion
        return f"${data / 1e9:.2f} B"
    elif data >= 1e6:  # Million
        return f"${data / 1e6:.2f} M"
    else:
        return f"${data:.2f}"

def extract_domain(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc

    if domain.startswith('www.'):
        domain = domain[4:]  # Exclude 'www.' prefix
    return domain
# Function to get company logo
def get_company_logo(ticker,df):
    website = df.loc[df['ticker'] == ticker, 'website'].values[0]
    domain = extract_domain(website)
    url = f"https://logo.clearbit.com/{domain}"

    resp = requests.get(url)
    
    logo_image = Image.open(BytesIO(resp.content))
    
    # Convert the image to have a transparent background
    logo_image = logo_image.convert("RGBA")
    data = logo_image.getdata()

    # Create a new image with a transparent background
    new_data = []
    for item in data:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            new_data.append((255, 255, 255, 0))  # Set white pixels to be transparent
        else:
            new_data.append(item)

    logo_image.putdata(new_data)
    
    return logo_image

def get_data_info(ticker_list): 
    # Split the list into tickers
    tickers = ticker_list.split(' ')

    # Create a list to accumulate data
    data_list = []

    for ticker in tickers:
        # Call function to get information
        company_info = yf.Ticker(ticker).info

        # Assigne the information
        company_name = company_info.get('longName', 'N/A')
        company_website = company_info.get('website', 'N/A')
        sector = company_info.get('sector', 'N/A')
        industry = company_info.get('industry', 'N/A')
        dividend_yield = company_info.get('dividendYield', 0) * 100
        dividend_rate= company_info.get('dividendRate','N/A')
        currency = company_info.get('currency','N/A')
        financialCurrency = company_info['financialCurrency']
        currency = financialCurrency if currency == 'NA' else currency
        market_cap = company_info.get('marketCap', 'N/A')
        trailing_pe= company_info.get('trailingPE','N/A')
        ebitda = company_info.get('ebitda','N/A')
        ebitda_margin = company_info.get('ebitdaMargins','N/A')
        profit_margins=  company_info.get('profitMargins','N/A')
        peg_ratio=  company_info.get('pegRatio','N/A')
        revenue=  company_info.get('totalRevenue','N/A')
        earning_growth=  company_info.get('earningsGrowth','N/A')
        revenue_growth=  company_info.get('revenueGrowth','N/A')


        # Append a new row to the list
        data_list.append({
            'company_name': company_name,
            'ticker': ticker,
            'website': company_website,
            'sector': sector,
            'industry': industry,
            'dividend_yield': dividend_yield,
            'currency' : currency, # financialCurrency
            'market_cap': market_cap,
            'trailing_pe': trailing_pe,
            'ebitda': ebitda,
            'ebitda_margin': ebitda_margin,
            'profit_margins': profit_margins,
            'peg_ratio': peg_ratio,
            'revenue': revenue,
            'earning_growth': earning_growth,
            'revenue_growth': revenue_growth,
        })

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data_list)
    # Apply the conversion function to the 'revenue' and 'market_cap' columns
    df = df.apply(convert_to_usd, axis=1)
    # Conver to numeric
    exclude_columns = ['company_name', 'ticker','website','sector','industry']
    transform_all_except_specified(df, exclude_columns)
    # Add format to Market cap and Revenue
    df['formatted_market_cap'] = df['market_cap'].apply(format_decimal_subscript)
    df['formatted_revenue'] = df['revenue'].apply(format_decimal_subscript)

    return df

def compare_two_companies(ticker_list):
    # Assuming df is the DataFrame returned by get_data_info function
    df = get_data_info(ticker_list)
    
    # External CSS for dark mode theme
    external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
    app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

    # Create app layout
    app.layout = html.Div(children=[
        dcc.Location(id='url', refresh=False),
        
        # Centered title
        html.H1("Companies Comparison", style={'text-align': 'center'}),

        # Two columns for company comparison
        html.Div([
            # Company 1
            html.Div([
                html.H4(f"{df.iloc[0]['company_name']}"),
                
                # Add company logo
                html.P([
                    html.Img(src=get_company_logo(df.iloc[0]['ticker'],df), style={'width': '100px', 'height': '100px'}),
                ]),
                html.P([
                    f"{df.iloc[0]['sector']}"
                ]),
                html.P([
                    f"{df.iloc[0]['industry']}"
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[0]['formatted_market_cap']) or str(df.iloc[0]['formatted_market_cap']).lower() == 'nan' else f"{df.iloc[0]['formatted_market_cap']}"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['formatted_market_cap']) or df.iloc[1]['formatted_market_cap'] == 'nan' else ('grey' if df.iloc[0]['formatted_market_cap'] < df.iloc[1]['formatted_market_cap'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[0]['formatted_revenue']) or str(df.iloc[0]['formatted_revenue']).lower() == 'nan' else f"{df.iloc[0]['formatted_revenue']}"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['formatted_revenue']) or df.iloc[1]['formatted_revenue'] == 'nan' else ('grey' if df.iloc[0]['formatted_revenue'] < df.iloc[1]['formatted_revenue'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[0]['earning_growth']) or str(df.iloc[1]['earning_growth']).lower() == 'nan' else f"{df.iloc[0]['earning_growth']*100:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['earning_growth']) or df.iloc[0]['earning_growth'] == 'nan' else ('grey' if df.iloc[0]['earning_growth'] < df.iloc[1]['earning_growth'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[0]['dividend_yield']) or str(df.iloc[0]['dividend_yield']).lower() == 'nan' else f"{df.iloc[0]['dividend_yield']:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['dividend_yield']) or df.iloc[1]['dividend_yield'] == 'nan' else ('grey' if df.iloc[0]['dividend_yield'] < df.iloc[1]['dividend_yield'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[0]['profit_margins']) or str(df.iloc[1]['profit_margins']).lower() == 'nan' else f"{df.iloc[0]['profit_margins']*100:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['profit_margins']) or df.iloc[0]['profit_margins'] == 'nan' else ('grey' if df.iloc[0]['profit_margins'] < df.iloc[1]['profit_margins'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[0]['ebitda_margin']) or str(df.iloc[1]['ebitda_margin']).lower() == 'nan' else f"{df.iloc[0]['ebitda_margin']*100:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['ebitda_margin']) or df.iloc[0]['ebitda_margin'] == 'nan' else ('grey' if df.iloc[0]['ebitda_margin'] < df.iloc[1]['ebitda_margin'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[0]['trailing_pe']) or str(df.iloc[0]['trailing_pe']).lower() == 'nan' else f"{df.iloc[0]['trailing_pe']:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['trailing_pe']) or df.iloc[1]['trailing_pe'] == 'nan' else ('grey' if df.iloc[0]['trailing_pe'] < df.iloc[1]['trailing_pe'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                # Add other KPIs for Company 1 here
            ], style={'width': '35%', 'display': 'inline-block', 'margin-right': '2%', 'text-align': 'center'}),

            # Middle column
            html.Div([
                
                # Add company logo
                html.P([
                    html.H3(" VS "),
                ]),
                html.P([
                    html.Strong("Sector"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("Industry"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("Market Cap"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("Revenue"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("Earnings Growth"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("Dividend Yield"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("Profit Margins"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("EBITA margins"),
                    html.Br(),
                ]),
                html.P([
                    html.Strong("PE Ratio"),
                    html.Br(),
                ]),
                # Add other KPIs for Company 1 here
            ], style={'width': '20%', 'display': 'inline-block', 'margin-right': '2%', 'text-align': 'center'}),

            # Company 2
            html.Div([
                html.H4(f"{df.iloc[1]['company_name']}"),
                
                # Add company logo
                html.P([
                    html.Img(src=get_company_logo(df.iloc[1]['ticker'],df), style={'width': '100px', 'height': '100px'}),
                ]),
                html.P([
                    f"{df.iloc[1]['sector']}",
                ]),
                html.P([
                    f"{df.iloc[1]['industry']}"
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[1]['formatted_market_cap']) or str(df.iloc[1]['formatted_market_cap']).lower() == 'nan' else f"{df.iloc[1]['formatted_market_cap']}"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['formatted_market_cap']) or df.iloc[1]['formatted_market_cap'] == 'nan' else ('grey' if df.iloc[0]['formatted_market_cap'] > df.iloc[1]['formatted_market_cap'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[1]['formatted_revenue']) or str(df.iloc[1]['formatted_revenue']).lower() == 'nan' else f"{df.iloc[1]['formatted_revenue']}"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['formatted_revenue']) or df.iloc[1]['formatted_revenue'] == 'nan' else ('grey' if df.iloc[0]['formatted_revenue'] > df.iloc[1]['formatted_revenue'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[1]['earning_growth']) or str(df.iloc[1]['earning_growth']).lower() == 'nan' else f"{df.iloc[1]['earning_growth']:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['earning_growth']) or df.iloc[1]['earning_growth'] == 'nan' else ('grey' if df.iloc[0]['earning_growth'] > df.iloc[1]['earning_growth'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[1]['dividend_yield']) or str(df.iloc[1]['dividend_yield']).lower() == 'nan' else f"{df.iloc[1]['dividend_yield']:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['dividend_yield']) or df.iloc[1]['dividend_yield'] == 'nan' else ('grey' if df.iloc[0]['dividend_yield'] > df.iloc[1]['dividend_yield'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[1]['profit_margins']) or str(df.iloc[1]['profit_margins']).lower() == 'nan' else f"{df.iloc[1]['profit_margins']*100:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['profit_margins']) or df.iloc[1]['profit_margins'] == 'nan' else ('grey' if df.iloc[0]['profit_margins'] > df.iloc[1]['profit_margins'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[1]['ebitda_margin']) or str(df.iloc[1]['ebitda_margin']).lower() == 'nan' else f"{df.iloc[1]['ebitda_margin']*100:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['ebitda_margin']) or df.iloc[1]['ebitda_margin'] == 'nan' else ('grey' if df.iloc[0]['ebitda_margin'] > df.iloc[1]['ebitda_margin'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                html.P([
                    html.Span(
                        f"{'-' if pd.isna(df.iloc[1]['trailing_pe']) or str(df.iloc[1]['trailing_pe']).lower() == 'nan' else f"{df.iloc[1]['trailing_pe']:.2f}%"}",
                        style={
                            'color': 'black' if pd.isna(df.iloc[1]['trailing_pe']) or df.iloc[1]['trailing_pe'] == 'nan' else ('grey' if df.iloc[0]['trailing_pe'] > df.iloc[1]['trailing_pe'] else 'black'),
                            'font-family': 'Sans-Serif'
                        }
                    )
                ]),
                # Add other KPIs for Company 2 here
            ], style={'width': '35%', 'display': 'inline-block','text-align': 'center'}),
        ]),

        # Add other KPIs or graphs for comparison here
    ])

    if __name__ == '__main__':
        app.run_server(debug=True)

In [15]:
# Place the two ticker of companies to compare
compare_two_companies("OLN COIHY")